# Prototype Version 4
This code will read data from an tsv file, this file contains associations of the name of a tandem spectra and their corresponding amino acids sequence. I will find the corresponding spectra in an mzML file and create a Dataframe from matches.
<br>
This code will then find the shortest spectra <b>visualize</b> it and <b>identify</b> the presumably correct peaks based on the match.
This programs purpose is to recreate this spectra annotation as faithfully as possible just based on the distances between the peaks (which are assumed to be y-ions). The code will <b>recognize</b> amino acids that could fit in between these peaks.
<br>
This code will create a distance matrix and score every single distance between peaks and which of them could fit an amino acids or pair of amino acids between them. Using this distance matrix the code will find every single possible peptide. These are then given a confidence score referring to their likelyhood of being the true peptide. This confidence score is adapted from the 2003 paper by David Tabb named DirecTag https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2810657/
<br>
This approach of identifying peptides is called <b>DeNovo </b> and this code is based on a presentation by Nick Webb (https://www.weddslist.com/ms/tandem.html)

I will investigate how well my programs predict the amino acid sequence by comparing the actual vs predicted sequence AND how well my confidence score works.

At the end, in order to check my results, I will use the pyteomics.pylab_aux module to test my annotations and visualize the

In [ ]:
import ast

# basic dependencies and useful math/organization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# to read mzML files
from pyteomics import mzml

# to visualize mzML
from pyteomics import pylab_aux as pa, usi

# to find peaks
from scipy.signal import find_peaks
from peakutils import indexes

## reading the files

In [ ]:
# file that contains the associations between tandem spectra name and amino acids sequence
df = pd.read_csv("../data/psm_a.tsv", sep="\t")

In [ ]:
# file that contains the amino acids and their weights
# TODO for now there is a distinction between "I" and "L"
# They are indistinguishable by weight so I cant really know which one it is
aa = pd.read_csv("../data/single_double_amino_acids.csv")

In [ ]:
# this creates a dictionary of matches between the mzMl file and the psm_a.tsv
# this should only return MS/MS where we have the Peptide sequence already identified.
i = 0
matches = []
plotting_dict = None
with mzml.MzML(mz_path) as reader:
    for spectrum in reader:
        # looking for the first match between the 2 files
        for name in df["Spectrum"]:
            if name in spectrum.get('spectrum title') :
                # print(spectrum["index"], spectrum.get("spectrum title"))
                # Extract relevant information
                matches.append([spectrum,df[df["Spectrum"] == name]["Peptide"].sum(),
                               df[df["Spectrum"] == name]["Hyperscore"].sum()])
                """
                spectrum_name = name
                mz_array = spectrum['m/z array']
                intensity_array = spectrum['intensity array']
                precursor_mz = (spectrum['precursorList']['precursor']
                                [0]['selectedIonList']['selectedIon'][0]['selected ion m/z'])

                plotting_dict = {'m/z array': mz_array, 'intensity array': intensity_array}
                """
        #         break
        # if plotting_dict != None:
        #     break
m_df = (pd.DataFrame(matches, columns=["Spectrum","Peptide","Hyperscore"])
    .sort_values(by = ["Hyperscore"], axis=0, ascending= False)
    .reset_index(drop=True))


## deciding on which Peptide sequence to use
I am going to aim for the shortest one for simplicity

In [ ]:
print(min(m_df["Peptide"], key = len))
print(m_df[m_df["Peptide"] == str(min(m_df["Peptide"], key = len))])
pep_loc = 7103

In [ ]:
# collecting the data in variables
peptide = m_df["Peptide"].iloc[pep_loc]
print("Peptide sequence:", peptide)
mz_array = m_df["Spectrum"].iloc[pep_loc]['m/z array']
intensity_array = m_df["Spectrum"].iloc[pep_loc]['intensity array']
precursor_mz = m_df["Spectrum"].iloc[pep_loc]['precursorList']['precursor'][0]['selectedIonList']['selectedIon'][0]['selected ion m/z']
plotting_dict = {'m/z array': mz_array, 'intensity array': intensity_array}


## Finding Peaks
using sci_peak

In [ ]:
s1 = pd.DataFrame(plotting_dict)
pa.plot_spectrum(plotting_dict)
plt.figure(figsize=(12,6))
plt.show()

In [ ]:
sp_height = intensity_array.max()*0.05
sp_dist = None
sp_prom = None
#TODO find ways to optimize for these parameters, possible based on characteristics of each spectrum
# 5% of max height should be the cutoff threshold
# i.e.: based on how long we expect the amino acid sequence to be
# what determines wheter a peak is useful, is there someway we can find out?
#

sci_peak, _ = find_peaks(s1["intensity array"],
                      height=sp_height,
                      distance=sp_dist,
                      prominence=sp_prom
                      )
sns.set_theme(rc={'figure.figsize':(20,6)})
sns.scatterplot(data = s1.iloc[sci_peak].reset_index(),
                x = "m/z array",
                y = "intensity array",
                color = "red", alpha = 0.5).set_xticks(np.arange(0,1200,50))
pa.plot_spectrum(plotting_dict)
#plt.plot(np.zeros_like(x), "--", color="gray")
#plt.figure(figsize=(12,6))
plt.show()

## automatically annotating the spectra with pyteomics
based on the "real" peptide

In [ ]:
pa.annotate_spectrum(plotting_dict, peptide, precursor_charge=2, backend='spectrum_utils',
    ion_types='aby', title=peptide)
#plt.figure(figsize=(12,6))
plt.show()
len(peptide)

## Function for finding the closes amino acids

In [ ]:
# cheesing an issue with the apply function
# TODO pass this as an argument for the function instead of letting it access an outside variable
combo_df = aa

In [ ]:
import math
def find_closest_aa(value, thres = 2.5):
    """ Find the closest amino acid to a given mass.
    this code takes a dataframe with single amino acids and combinations of amino acids in this format

        full    letter  short   comp    mono    mass    G           A           S           P   V etc.
    0   glycine G       gly     c2h3no  57      57      g+g mass    g+a mass    g+s mass
    1   alanine A       ala     c3h5no  71      71      a+g mass    a+a mass    a+s mass
    etc.

    using this dataframe and a given mass, it will find the closest amino acid
     or combination of 2 amino acids to the given mass.
    It will only return amino acids that are within the threshold.
    It returns a list of lists, where each list contains the letter of the amino acid, its mass and the error.
    """
    # TODO remove useless calculations from this like closeness_list having values that I never use
    if math.isnan(value) == True:
        return None
    single_df = combo_df.iloc[:,:-22]
    double_df = combo_df.set_index(["letter"]).iloc[
                :,[i for i in range(-22, -0)]]
    closeness_list = []
    # find the closest single amino acids
    loop_for_single = True
    while loop_for_single:
        aam_array = np.asarray(single_df["mono mass"])
        idx = (np.abs(aam_array - value)).argmin()
        error = np.abs(aam_array[idx] - value)
        if error > thres:
            loop_for_single = False
        else:
            name_idx = single_df["letter"].iloc[idx]
            closeness_list.append([name_idx, aam_array[idx], error])
            single_df = single_df.drop(single_df.index[idx])
    # find closest combination of amino acids
    loop_for_combo = True
    while loop_for_combo:
        error = (np.abs(double_df - value)).min().min()
        r, c = np.where(double_df == error + value)
        # if error wouldve been negative np.where will not find r, c
        # and pass empty arrays creating error
        if r.size == 0 :
            # print("boink")
            r, c = np.where(double_df == value - error)
        if error > thres:
            loop_for_combo = False
        else:
            name_idx = double_df.index[r[0]]+ "+" + double_df.columns[c[0]]
            closeness_list.append([name_idx, double_df.iloc[r[0],c[0]], error])
            double_df.iloc[r[0],c[0]] = None
    # print("closest aa is: ", name_idx, " ,with mass: ",
    # aam_array[idx], "Da. With an error of: ", error, "Da.")
    closeness_list.sort(key=lambda x: x[2])
    if closeness_list:
        # return closeness_list[0]
        return closeness_list[0][0]
    return None


'''
I would love to decided a closeness factor by iterating over the weights
and seeing how close they are to each other on average. Does this make sense?
'''

# Creating the distance Matrix

In [ ]:
mz_peaks = mz_array[sci_peak]
intensity_peaks = intensity_array[sci_peak]
mz_round = np.round(mz_peaks, decimals= 6)

In [ ]:
# on my stackoverflow someone suggested a better way to create my distance matrix
# however I am now so deep in this going back would cost too much time

# this code builds a distance matrix for the given mz_array
# unless there is already a file in the directory it can just read

def rename(column):
    new = mz_round[int(column)]
    return new

try:
    df_aa = pd.read_csv("below_shortest_example_matrix_named.csv").set_index("Unnamed: 0")
except:
    distance_matrix = []
    for column, i in zip(mz_round, range(len(mz_round))):
        distance_matrix.append([])
        for row in mz_round:
            distance_matrix[i].append(np.abs(row - column))

    df_dm = pd.DataFrame(distance_matrix)
    df_dm.columns = df_dm.columns.map(str)

    # for i in range(len(mz_round)):
    #     for j in range(i):
    #         df_dm.iat[i,j] = None
    for i in range(len(mz_round)):
        for j in range(len(mz_round)):
            if j >= i:
                df_dm.iat[i,j] = None
            if df_dm.iat[i,j] < 55 or df_dm.iat[i,j] > 480:
                df_dm.iat[i,j] = None

    # if j <= i:
    #     df_dm.iat[i,j] = None
    # drop columns with no values (all NaN)
    df_dm.dropna(axis = 1, how="all", inplace = True)

    print("starting mapping")
    df_aa = df_dm.map(find_closest_aa)
    print("mapping complete")

    df_aa.rename(columns=rename, inplace=True)
    df_aa.rename(index=rename, inplace=True)

    df_aa.to_csv("below_shortest_example_matrix_named.csv")

    #have to reread so dictionaries are reconverted into strings
    # this is stupid but makes the code work consistently with read csv files
    df_aa = pd.read_csv("below_shortest_example_matrix_named.csv").set_index("Unnamed: 0")


## function to find all possible paths through this distance matrix

In [ ]:
# TODO give this function somehow sensible arguments to pass
# kids look away this function is not greatly formatted
def find_all_paths(n = 1000):
    # This is written to work form the highest peak down to the lowest similar to how its done by nick wedd.
    # however my distance matrix removed all points below the diagonal meaning anytime I identify a hit of an amino acid it looks like follows
    # hit a row: x and column: y. where y > x and since my program iterates over rows and then columns it sets the column to be the new row.
    # which means it starts back at the top and repeats itself in a pattern. Ill have to change the orientation of the matrix.
    full_pep_list = []

    blacklist = []

    for j in range(n):
        path_one = []
        peaks_one = []
        cur_peak = float(mz_round[-1])
        #
        while cur_peak > float(mz_round[0]):
            found = False
            for column in reversed(df_aa.columns):
                # print("conk", type(aa))
                peak_pos = np.where(mz_round == cur_peak)[0][0]
                aa_inf = df_aa[column].iloc[peak_pos]#.values[0]
                aa_int = intensity_peaks[peak_pos]
                # print("row:",mz_round[peak_pos],"column: ", column,"aa:",aa_inf,"int:",aa_int)
                #time.sleep(0.1)
                # choosing the first possible connection always as of now
                if isinstance(aa_inf, str) and peaks_one + [mz_round[peak_pos], float(column)] not in blacklist:
                    # and peaks_one not in blacklist:# or float(column) not in Xblacklist):
                    # print("\n boingoloingo \n",mz_round[peak_pos],"\n",column," \n")
                    found = True
                    # if peaks_one + [mz_round[peak_pos], float(column)] in blacklist:
                    #     found = False
                    #     print("WE HAVE BEEN DOWN THIS PATH BEFORE, ABORT MISSION AJDKSDFSDKFJ")
                    #     break
                    path_one.append(aa_inf)
                    peaks_one.append(mz_round[peak_pos])
                    #peaks_one.append(mz_round[peak_pos])
                    # ast.literal_eval is a security risk but I dont know a better way
                    # Problem is that dictionaries get saved as string when .to_csv is applied
                    cur_peak = float(column)
                    last_peak = float(column)
            if not found:
                # print("skip to next row")
                cur_peak = float(mz_round[peak_pos-1])
                found = False

        peaks_one.append(last_peak)
        blacklist.append(peaks_one)
        # info_dic = {}
        # for i in range(len(peaks_one)):
        #     info_dic[i] = peaks_one[i]
        full_pep_list.append([path_one, peaks_one])
        #print("new peptide finished")
        #print(path_one)

    return full_pep_list

In [ ]:
find_all_paths()

In [ ]:
pd.DataFrame(full_pep_list)

In [ ]:
# TODO find a way to compute every single possible peptide
# this will create soooo many peptides tho
# TODO implement the DirecTag ranking methods.
# TODO is there a heuristic for this???
# like can I find a way to know ahead of time which path makes the most sense

## implement confidence scoring function

## apply confidence score

## distribution of confidence scores

## visualize most confident peptide and its MS/MS